In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np

import plotly.graph_objects as go
import cufflinks as cf
cf.set_config_file(offline=True, sharing=False, theme='ggplot')

from sklearn.model_selection import cross_val_score

# Introduction to Lasso and Ridge Regression

Linear regression aims to find the best-fitting line (or hyperplane in higher dimensions) that represents the relationship between a dependent variable (target) and one or more independent variables (features). It does this by minimizing the sum of squared errors (or another loss) between the predicted values and the actual values.

Overfitting: A Common Challenge:
- Overfitting occurs when a model learns the training data too well, including its noise and outliers. This leads to excellent performance on the training data but poor generalization to unseen data.

Regularization: Lasso and Ridge to the Rescue
- Lasso (L1 regularization) and Ridge (L2 regularization) are techniques that add penalty terms to the ordinary least squares (OLS) cost function of linear regression. These penalties discourage overly complex models and help prevent overfitting.

# Lasso Regression (L1 Regularization)
Lasso adds a penalty term proportional to the absolute value of the coefficients. It is particularly useful when you have a large number of features, some of which are not useful. This penalty can force some coefficients to become exactly zero, effectively performing feature selection.
\begin{equation}
 \sum_{i=1}^{n} \left( y_i - \sum_{j=1}^{p} x_{ij}\beta_j \right)^2 + \lambda \sum_{j=1}^{p} |\beta_j|
\end{equation}


# Ridge Regression (L2 Regularization)
Ridge regression adds a penalty term proportional to the square of the coefficients. This penalty shrinks the coefficients towards zero, reducing their impact but generally not eliminating them entirely.
\begin{equation}
\sum_{i=1}^{n} \left( y_i - \sum_{j=1}^{p} x_{ij}\beta_j \right)^2 + \lambda \sum_{j=1}^{p} \beta_j^2
\end{equation}

# Example: Fuel Mileage



In [1]:
from seaborn import load_dataset
data = load_dataset("mpg")
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


Let us create a train/test split.

In [4]:
train_set, test_set = train_test_split(data, test_size=0.25, random_state=83)

In the following cells, we will test linear regression models with an increasing number of features, and observe the training RMSE, cross-validation RMSE and test RMSE as the number of features is increased. 

The keys of the `models` dictionary are built based on the first letters of the features included in that model.

In [6]:
quantitative_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year"]

models = {}
for i in range(len(quantitative_features)):
    # The features to include in the ith model
    features = quantitative_features[:(i+1)]
    # The name we are giving to the ith model
    name = ",".join([name[0] for name in features])
    # The pipeline for the ith model
    model = Pipeline([
        ("SelectColumns", ColumnTransformer([
            ("keep", "passthrough", features),
        ])),
        ("Imputation", SimpleImputer()),
        ("LinearModel", LinearRegression())
    ])
    # Fit the pipeline
    model.fit(train_set, train_set['mpg'])
    # Saving the ith model
    models[name] = model
    
# The models we have trained
models.keys()

dict_keys(['c', 'c,d', 'c,d,h', 'c,d,h,w', 'c,d,h,w,a', 'c,d,h,w,a,m'])

## Adding the Origin as a Feature

The origin represents the country of origin of the vehicle. Since this is a categorical feature, we will use scikit-learn's `OneHotEncoder` to translate that into a numerical feature that can be used in a regression model. 

In [11]:
train_set['origin'].value_counts()

origin
usa       188
japan      56
europe     54
Name: count, dtype: int64

In [15]:
from sklearn.preprocessing import OneHotEncoder

model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"])
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

model.fit(train_set, train_set['mpg'])
name = ",".join([name[0] for name in quantitative_features]) + ",o"
models[name] = model

## Adding the Vehicle Name as a Feature

The vehicle name is also available in our dataset and can be used as a feature. 


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

model.fit(train_set, train_set['mpg'])
name = ",".join([name[0] for name in quantitative_features]) + ",o,n"
models[name] = model

Using the following `compare_models`, we will plot the RMSE score for each model and use the score to evaluate each model.

In [17]:
def rmse_score(model, X, y):
    return np.sqrt(np.mean((y - model.predict(X))**2))

def compare_models(models):
    # Compute the training error for each model
    training_rmse = [rmse_score(model, train_set, train_set['mpg']) for model in models.values()]
    # Compute the cross validation error for each model
    validation_rmse = [np.mean(cross_val_score(model, train_set, train_set['mpg'], scoring=rmse_score, cv=5)) for model in models.values()]
    # Compute the test error for each model (don't do this!)
    test_rmse = [rmse_score(model, test_set, test_set['mpg']) for model in models.values()]
    names = list(models.keys())
    fig = go.Figure([
        go.Bar(x=names, y=training_rmse, name="Training RMSE"),
        go.Bar(x=names, y=validation_rmse, name="CV RMSE"),
        go.Bar(x=names, y=test_rmse, name="Test RMSE", opacity=.3)
			])
    fig.update_yaxes(title="RMSE")
    return fig


compare_models(models)

For models `c` through `c,d,h,w,a,m,o`, we see that the training, cross-validation and test RMSE scores decrease monotonically as features are added. However when moving to `c,d,h,w,a,m,o,n` there is a dramatic drop in the training RMSE, yet the cross-validation RMSE increases - this is indicative of overfitting.

To understand why this might have happened, lets take a look at the number of features in each model:

In [28]:
for m in models:
	print(f"Model: {m:<20} Features: {len(models[m]['LinearModel'].coef_)}")

Model: c                    Features: 1
Model: c,d                  Features: 2
Model: c,d,h                Features: 3
Model: c,d,h,w              Features: 4
Model: c,d,h,w,a            Features: 5
Model: c,d,h,w,a,m          Features: 6
Model: c,d,h,w,a,m,o        Features: 9
Model: c,d,h,w,a,m,o,n      Features: 270


Thus we see that adding in the vehicle name as a feature drastically increased the feature count. This is a natural point at which to begin considering regularization methods to see if the cross-validation error can be reduced further.



## A Ridge Regression Model

In [31]:
from sklearn.linear_model import Ridge

ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", Ridge(alpha=0.5))
])

ridge_model.fit(train_set, train_set['mpg'])
models["Ridge(alpha=0.5)"] = ridge_model
compare_models(models)

In the plot above we see that the training RMSE did increase, yet we were also able to address the overfitting and reduce the cross validation error.

## Standardizing Features

In the context of Ridge regression, standardizing our features is important. 

For example, weight can vary on the order of thousands, and feautres like origin which is one-hot encoded will have a range of values form 0 to 1. Thus applying a single regularization parameter to all of these features, which vary on different ranges will have a disproportionate impact on the larger features like the weight.

The `StandardScaler` class of scikit-learn will subtract the mean from each column and divide by that column's standard deviation.

In [32]:
from sklearn.preprocessing import StandardScaler

ridge_model = Pipeline([
	("SelectColumns", ColumnTransformer([
		("keep", StandardScaler(), quantitative_features),
		("origin_encoder", OneHotEncoder(), ["origin"]),
		("text", CountVectorizer(), "name")
	])),
	("Imputation", SimpleImputer()),
	("LinearModel", Ridge(alpha=0.5))
])

ridge_model.fit(train_set, train_set['mpg'])
models["RidgeN(alpha=0.5)"] = ridge_model
compare_models(models)

By standardizing the quantiative features, we were able to get another slight reduction in the cross-validation error.

## Using Cross-Validation to find an Optimal value for our Regularization Hyperparameter

One option for finding the optimal value for the regularization parameter $\alpha$ is to simply try out a range of different values for $\alpha$ and visually or numerically find the value that minimises the cross-validation error. This can be done manually of course, however this unnecessary since scikit-learn has a ridge regression class with cross-validation built in: `RidgeCV`.

In [33]:
from sklearn.linear_model import RidgeCV

alphas = np.linspace(0.5, 3, 30)

ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", StandardScaler(), quantitative_features),
        ("origin_encoder", OneHotEncoder(), ["origin"]),
        ("text", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", RidgeCV(alphas=alphas))
])

ridge_model.fit(train_set, train_set['mpg'])
models["RidgeCV"] = ridge_model
compare_models(models)

This model achieves the best cross-validation error yet:

In [36]:
np.mean(cross_val_score(models["RidgeCV"], train_set, train_set['mpg'], scoring=rmse_score, cv=5))

2.9630429994240512

The optimal value of $\alpha$ found by scikit-learn can be displayed as follows.

In [34]:
models["RidgeCV"]['LinearModel'].alpha_

1.706896551724138

# Conclusion

Lasso and Ridge regression are powerful techniques for improving the generalization performance
of linear regression models. By adding penalty terms to the cost function, they help prevent
overfitting and can lead to more robust and reliable predictions.

Further, scikit-learn's `RidgeCV` provides a robust and concise way to find an optimal regularization parameter when using Ridge regression.